# **Freshness Analysis**

# Import neccessary dependencies

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


# Define paths to the training and test data

In [3]:

train_data_path = r'C:\Users\harsh\Downloads\meena\archive\dataset\Train'
test_data_path = r'C:\Users\harsh\Downloads\meena\archive\dataset\Test'
    

# Data augmentation with ImageDataGenerator

In [4]:

train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    validation_split=0.2 
)

# Training and validation generators
train_generator = train_datagen.flow_from_directory(
    train_data_path,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    train_data_path,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)
    

Found 12355 images belonging to 8 classes.
Found 3084 images belonging to 8 classes.


# Test data generator

In [5]:

test_datagen = ImageDataGenerator(rescale=1.0 / 255)  # Only rescale for test data
test_generator = test_datagen.flow_from_directory(
    test_data_path,
    target_size=(150, 150),
    batch_size=64,
    class_mode='categorical',
    shuffle=False  # Important to keep the order for evaluation
)
    

Found 4713 images belonging to 8 classes.


# Print class indices to confirm number of classes

In [6]:


print("Class indices: ", train_generator.class_indices)
    

Class indices:  {'freshapples': 0, 'freshbanana': 1, 'freshbittergroud': 2, 'freshtomato': 3, 'rottenapples': 4, 'rottenbanana': 5, 'rottenbittergroud': 6, 'rottentomato': 7}


In [8]:

# Load VGG16 model + higher level layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Freeze the base model
base_model.trainable = False

# Create the hybrid model
model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(train_generator.class_indices), activation='softmax')  # Output layer
])
    

# Compile the model

In [9]:


model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    

# Callbacks for early stopping and learning rate reduction

In [10]:


early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, min_lr=1e-6)
    

# Train the model

In [11]:


model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=30,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    callbacks=[early_stopping, reduce_lr]
)
    

Epoch 1/30
386/386 [==============================] - 67s 153ms/step - loss: 0.3001 - accuracy: 0.8980 - val_loss: 0.7247 - val_accuracy: 0.7640 - lr: 0.0010
Epoch 2/30
386/386 [==============================] - 61s 159ms/step - loss: 0.0985 - accuracy: 0.9666 - val_loss: 0.6695 - val_accuracy: 0.7965 - lr: 0.0010
Epoch 3/30
386/386 [==============================] - 63s 162ms/step - loss: 0.0742 - accuracy: 0.9741 - val_loss: 0.7269 - val_accuracy: 0.7969 - lr: 0.0010
Epoch 4/30
386/386 [==============================] - 62s 160ms/step - loss: 0.0660 - accuracy: 0.9754 - val_loss: 0.6612 - val_accuracy: 0.8089 - lr: 0.0010
Epoch 5/30
386/386 [==============================] - 65s 168ms/step - loss: 0.0561 - accuracy: 0.9805 - val_loss: 0.8077 - val_accuracy: 0.7952 - lr: 0.0010
Epoch 6/30
386/386 [==============================] - ETA: 0s - loss: 0.0656 - accuracy: 0.9767
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
386/386 [=============================

In [12]:

# Evaluate the model on the validation set
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f'Validation Accuracy: {val_accuracy:.2f}')

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test Accuracy: {test_accuracy:.2f}')
    

97/97 [==============================] - 16s 160ms/step - loss: 0.6601 - accuracy: 0.8087
Validation Accuracy: 0.81
74/74 [==============================] - 39s 435ms/step - loss: 1.0993 - accuracy: 0.8835
Test Accuracy: 0.88


# Fine-tuning the model

In [13]:

# Unfreeze the last few layers of the base model
base_model.trainable = True
for layer in base_model.layers[:-4]:  # Unfreeze the last 4 layers
    layer.trainable = False

# Recompile the model with a lower learning rate
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Continue training for additional epochs
model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=15,  # Fewer epochs for fine-tuning
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    callbacks=[early_stopping, reduce_lr]
)

# Final evaluation on the test set after fine-tuning
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Final Test Accuracy after fine-tuning: {test_accuracy:.2f}')
    

Epoch 1/15
386/386 [==============================] - 75s 189ms/step - loss: 0.1281 - accuracy: 0.9592 - val_loss: 0.7613 - val_accuracy: 0.7868 - lr: 1.0000e-04
Epoch 2/15
386/386 [==============================] - 68s 177ms/step - loss: 0.0348 - accuracy: 0.9884 - val_loss: 0.8936 - val_accuracy: 0.7832 - lr: 1.0000e-04
Epoch 3/15
386/386 [==============================] - ETA: 0s - loss: 0.0362 - accuracy: 0.9897
Epoch 3: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-05.
386/386 [==============================] - 70s 181ms/step - loss: 0.0362 - accuracy: 0.9897 - val_loss: 0.8489 - val_accuracy: 0.7979 - lr: 1.0000e-04
Epoch 4/15
386/386 [==============================] - 71s 185ms/step - loss: 0.0105 - accuracy: 0.9962 - val_loss: 1.0026 - val_accuracy: 0.8324 - lr: 5.0000e-05
Epoch 5/15
386/386 [==============================] - ETA: 0s - loss: 0.0076 - accuracy: 0.9974
Epoch 5: ReduceLROnPlateau reducing learning rate to 2.499999936844688e-05.
386/386 [==========

In [14]:
from tensorflow.keras.models import load_model

# Assume 'model' is your trained model
model.save('trainedModel.h5')


In [15]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def classify_image(image_path, model_path):
    # Load the pre-trained model
    model = load_model(model_path)
    
    # Load the image file, resizing it to the expected input size of the model (150, 150 in your case)
    img = load_img(image_path, target_size=(150, 150))  # Adjusted size here
    
    # Convert the image to an array and expand dimensions to be suitable for model input
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Normalize the image array if required (depending on how the model was trained)
    img_array /= 255.0  # Scale the pixel values to [0, 1] if needed
    
    # Predict the class of the image
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction, axis=1)
    
    # Return the predicted class; you might want to adjust this to return a string label
    return predicted_class


# Example usage:

In [24]:
model_path = r'C:\Users\harsh\Downloads\meena\trainedModel.h5'
image_path = r'C:\Users\harsh\Downloads\meena\archive\dataset\Test\freshapples\vertical_flip_Screen Shot 2018-06-08 at 5.07.05 PM.png'


# Load the image file, resizing it to the expected input size of the model

In [25]:
img = load_img(image_path, target_size=(150, 150))


In [26]:
result = classify_image(image_path, model_path)
print("Predicted class:", result)

1/1 [==============================] - 0s 147ms/step
Predicted class: [0]
